In [1]:
import re
import numpy as np
import tensorflow as tf
from pyvi import ViTokenizer, ViPosTagger

# Enable Eager Execution
tf.enable_eager_execution()
tf.executing_eagerly() 


True

In [2]:
# Tạo word embeddings
words_list = np.load('data/words_list.npy')
words_list = words_list.tolist()
word_vectors = np.load('data/word_vectors.npy')
word_vectors = np.float32(word_vectors)

# Ma trận index của từ ánh xạ sang v
word2idx = {w: i for i, w in enumerate(words_list)}

In [3]:
class SentimentAnalysisModel(tf.keras.Model):
    """
    Mô hình phân tích cảm xúc của câu
    
    Properties
    ----------
    word2vec: numpy.array
        word vectors 
    lstm_layers: list
        list of lstm layers, lstm cuối cùng sẽ chỉ trả về output của lstm cuối cùng
    dropout_layers: list
        list of dropout layers
    dense_layer: Keras Dense Layer
        lớp dense layer cuối cùng nhận input từ lstm, 
        đưa ra output bằng số lượng class thông qua hàm softmax
    """
    def __init__(self, word2vec, lstm_units, n_layers, num_classes, dropout_rate=0.25):
        """
        Khởi tạo mô hình
        
        Paramters
        ---------
        word2vec: numpy.array
            word vectors 
        lstm_units: int
            số đơn vị lstm
        n_layers: int
            số layer lstm xếp chồng lên nhau
        num_classes: int
            số class đầu ra
        dropout_rate: float
            tỉ lệ dropout giữa các lớp
        """
        super().__init__(name='sentiment_analysis')
        
        # Khởi tạo các đặc tính của model
        self.word2vec = word2vec
        
        self.lstm_layers = []  # List chứa các tầng LSTM
        self.dropout_layers = []  # List chứa các tầng dropout

        ### TODO 3.1
        # Vòng lặp dưới chạy qua N tầng trong stack
        # mỗi tầng sẽ có 1 lstm_layer và 1 dropout_layer
        # 
        # Khởi tạo lstm_layer (GPU) với các tham biến sau:
        # lstm_layer(units=..., return_sequences=[True/False])
        # Tham khảo: https://keras.io/layers/recurrent/#cudnnlstm
        #
        # Khởi tạo lstm_layer (CPU) với các tham biến sau:
        # lstm_layer(units=..., return_sequences=[True/False])
        # Tham khảo: https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM
        # 
        # LƯU Ý:
        # return_sequence của lstm_layer nhận giá trị True ở mọi tầng  
        # trong stack, ngoại trừ tầng cuối cùngg
        #
        # Khởi tạo dropout_layer với các tham biến sau:
        # new_dropout = tf.keras.layers.Dropout(rate=...)
        # Tham khảo: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout
        # 
        # Sau khi khởi tạo lstm_layer và dropout_layer
        # hãy thêm chúng vào 2 list tương ứng
        # self.lstm_layers và self.dropout_layers
        #
        # Cuối cùng, khởi tạo tầng fully-connected/dense
        # tf.keras.layers.Dense(num_classes=..., activation=' ')
        # Tham khảo: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense
        #
        ### START CODE HERE
        
        for i in range(n_layers):
            new_lstm = tf.keras.layers.LSTM(units=lstm_units, return_sequences=True)
            self.lstm_layers.append(new_lstm)
            new_dropout = tf.keras.layers.Dropout(rate=dropout_rate)
            self.dropout_layers.append(new_dropout)
          
        #Tầng cuối cùng  
        new_lstm = tf.keras.layers.LSTM(units=lstm_units, return_sequences=False)
        self.lstm_layers.append(new_lstm)
        
        self.dense_layer = tf.keras.layers.Dense(num_classes, activation="softmax")
        ### END CODE HERE
        
    def call(self, inputs):
        ### TODO 3.2
        # Thực hiện các bước biến đổi khi truyền thuận input qua mạng
        # Điền code vào các đoạn pass
        ### START CODE HERE
        inputs = tf.cast(inputs, tf.int32)
        # Input hiện là indices, cần chuyển sang dạng vector
        # sử dụng:
        # tf.nn.embeddings_lookup(embeddings, indices)
        
        x = tf.nn.embedding_lookup(self.word2vec, inputs)
      
        # Truyền thuận inputs lần lượt qua các tầng
        # ở mỗi tầng, truyền input qua các layer: lstm > dropout
        # vd: x = first_lstm(x)
        #     x = first_dropout(x)
        #     x = second_lstm(x)
        #     v.v.
        n_layers = len(self.dropout_layers)
        
        for i in range(n_layers):
          x = self.lstm_layers[i](x)
          x = self.dropout_layers[i](x)
          
        x = self.lstm_layers[-1](x)
          
        x = self.dense_layer(x)
             
        # Gán giá trị tầng cuối cùng vào out và trả về
        out = x
        
        return out
        ### END CODE HERE
        

In [4]:
# Các hyperparameters
LSTM_UNITS = 128
N_LAYERS = 2
NUM_CLASSES = 2
MAX_SEQ_LENGTH = 200

In [5]:
strip_special_chars = re.compile("[^\w0-9 ]+")

def clean_sentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

In [6]:
def get_sentence_indices(sentence, max_seq_length, _words_list):
    """
    Hàm này dùng để lấy index cho từng từ
    trong câu (không có dấu câu, có thể in hoa)
    Parameters
    ----------
    sentence là câu cần xử lý
    max_seq_length là giới hạn số từ tối đa trong câu
    _words_list là bản sao local của words_list, được truyền vào hàm
    """
    indices = np.zeros((max_seq_length), dtype='int32')
    
    # Tách câu thành từng tiếng
    words = [word.lower() for word in sentence.split()]
    
    # Lấy chỉ số của UNK
    unk_idx = word2idx['UNK']
    
    ### TODO 1 ###
    # Viết lệnh cần thực hiện trong vòng lặp dưới [tgay vào pass]
    # Vòng lặp chạy qua tất cả các phần tử trong list `words`, tức các từ trong câu
    # vd: iter 1: idx = 1, word = tôi
    #     iter 2: idx = 2, word = đi
    #     iter 3: idx = 3, word = học
    #     iter 4: v.v
    # Trong mỗi vòng lặp, hãy thay các phần tử tương ứng
    # của `indices` bằng các chỉ số/index của các từ trong câu
    # LƯU Ý 1: len(indices) có thể ngắn hơn len(words)
    # LƯU Ý 2: câu có thể chứa những từ out of vocabulary (OOV), tức không có trong _word_list
    ### START CODE HERE ###
    for idx, word in enumerate(words):
        if idx < max_seq_length:
          if( word in _words_list):
            word_idx = word2idx[word]
          else:
            word_idx = word2idx['UNK']       
        
          indices[idx] = word_idx 
        else:
          break
          
    ### END CODE HERE ###
    return indices

In [7]:
def predict(sentence, model, _word_list=words_list, _max_seq_length=MAX_SEQ_LENGTH):
    """
    Dự đoán cảm xúc của một câu
    
    Parameters
    ----------
    sentence: str
        câu cần dự đoán
    model: model keras
        model keras đã được train/ load trọng số vừa train
    _word_list: numpy.array
        danh sách các từ đã biết
    _max_seq_length: int
        giới hạn số từ tối đa trong mỗi câu
        
    Returns
    -------
    int
        0 nếu là negative, 1 nếu là positive
    """
    ### TODO 7.1
    # Tokenize/Tách từ trong câu
    # Sử dụng hàm word_tokenize vừa import ở trên
    ### START CODE HERE
    tokenized_sent = ViTokenizer.tokenize(sentence)
    ### END CODE HERE
    
    ### TODO 7.2
    # Đưa câu đã tokenize về dạng input_data thích hợp để truyền vào model
    ### START CODE HERE
    tokenized_sent = clean_sentences(tokenized_sent)
    input_data = get_sentence_indices(tokenized_sent, _max_seq_length, _word_list)
    
    input_data = input_data.reshape(-1, _max_seq_length)
    ### END CODE HERE    
    
    ### TODO 7.3
    # Truyền input_data qua model để nhận về xác suất các nhãn
    # Chọn nhãn có xác suất cao nhất và return
    ### START CODE HERE
    pred = model(input_data)
    predictions =  tf.argmax(pred,1).numpy().astype(np.int32)
    return predictions
    ### END CODE HERE
    

In [8]:
model1 = SentimentAnalysisModel(word_vectors, LSTM_UNITS, N_LAYERS, NUM_CLASSES)

In [9]:
model1.load_weights(tf.train.latest_checkpoint('model'))

In [13]:
if predict("Quán này rất ngon.", model1) == 1:
    print('positive')
else:
    print('negative')
    
print("Test 2:",predict("Phim rất không hay chút nào.", model1))

positive
Test 2: [0]
